# vLLM 완벽 가이드: 이론부터 실전까지

이 노트북에서는 vLLM을 이용한 효율적인 LLM 추론을 단계별로 학습합니다.

## 학습 목표
1. ✅ vLLM의 핵심 개념 이해 (PagedAttention, KV Cache)
2. ✅ vLLM 기본 사용법 실습
3. ✅ LoRA adapter 통합 방법
4. ✅ Transformers vs vLLM 성능 비교
5. ✅ 실전 응용 (Text-to-SQL)

## 사전 준비
```bash
# uv 설치 (필요한 경우)
curl -LsSf https://astral.sh/uv/install.sh | sh

# 패키지 설치
uv pip install vllm==0.13.0 transformers torch peft
```

---
## Part 1: vLLM 기초 개념

### 1.1 왜 vLLM인가?

일반적인 Transformers 라이브러리의 문제점:
- **메모리 비효율**: KV Cache를 연속된 메모리에 저장 → 단편화 발생
- **느린 속도**: 배치 처리 시 비효율적
- **낮은 처리량**: 동시 요청 처리 능력 제한

vLLM의 해결책:
- **PagedAttention**: OS의 가상 메모리처럼 KV Cache를 블록 단위로 관리
- **연속 배칭**: 요청을 동적으로 배칭하여 처리량 극대화
- **최적화된 CUDA 커널**: GPU 활용도 향상

### 1.2 핵심 개념 1: PagedAttention (바인더 노트 비유)

코드를 보기 전에 딱 하나만 기억합시다.

**기존 방식 (스프링 노트)**:
- 페이지를 찢거나 중간에 끼워 넣을 수 없습니다
- 10장을 쓸지 모르고 100장짜리 노트를 사면 90장은 버리는 겁니다
- 연속된 메모리 필수 → 낭비 발생

```
KV Cache: [████████░░░░░░░░]  ← 미리 큰 메모리 할당, 낭비 발생
```

**PagedAttention (바인더/Loose-leaf 노트)**:
- 낱장(Block) 단위로 종이를 꺼내서 아무 데나 철해둡니다
- '목차(Block Table)'에 순서만 적어두면 됩니다
- 종이 낭비가 없습니다!

```
Block 0: [████]
Block 3: [████]  ← 필요할 때만 할당, 비연속적 저장 가능
Block 7: [██░░]
Block Table: [0, 3, 7]  ← 매핑 테이블로 관리
```

### 1.3 핵심 개념 2: KV Cache (일기장 비유)

**"나는 어제 학교에 가서 밥을 먹었다"** 라는 문장을 쓴다고 가정해봅시다.

**KV Cache가 없는 경우 (치매 걸린 작가)**:
1. "나는" (씀)
2. (기억 리셋) "나는" + "어제" (다시 읽고 씀)
3. (기억 리셋) "나는 어제" + "학교에" (다시 읽고 씀)
4. (기억 리셋) "나는 어제 학교에" + "가서" ...

→ **문제점**: 단어 하나 쓸 때마다 처음부터 끝까지 다시 계산. O(N²) 복잡도!

**KV Cache가 있는 경우 (스마트한 작가)**:
1. "나는" (씀. '나는'에 대한 정보를 포스트잇에 메모)
2. "어제" (아까 메모 확인, '어제' 정보 메모)
3. "학교에" (메모들 확인, '학교에' 정보 메모)

→ **핵심**: 과거 단어를 다시 계산하지 않고, 저장된 **메모(Key, Value)**만 참조. O(N) 복잡도!

**하지만 문제가 있습니다**:
- KV Cache가 계속 커짐 → GPU 메모리 부족 (OOM)
- **바로 여기서 PagedAttention이 등장!**
- "어차피 커질 `kv_cache`라면, 빈 공간 없이 쪼개서(Blocking) 효율적으로 저장하자"

In [ ]:
# 필요한 라이브러리 import
import os
import time
import torch
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

print("✅ 라이브러리 로드 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

---
## Part 1.5: PagedAttention 시뮬레이터 실습

이론만으로는 이해가 어렵습니다. 파이썬으로 **PagedAttention 시뮬레이터**를 만들어서 GPU 메모리 할당 과정을 직접 체험해봅시다!

### 시뮬레이터 구성요소
- **Logical Memory**: 사용자가 보는 문장 (연속됨)
- **Physical Memory**: 실제 데이터가 저장되는 곳 (뒤죽박죽 섞임)
- **Block Table**: 이 둘을 연결해주는 지도

In [ ]:
import random
from typing import List, Dict, Optional

class PhysicalBlock:
    """
    물리적인 메모리 블록 (GPU의 작은 조각)
    여기서는 편의상 하나의 블록에 4개의 토큰을 저장한다고 가정합니다 (BLOCK_SIZE = 4).
    """
    def __init__(self, block_id: int, block_size: int = 4):
        self.block_id = block_id
        self.size = block_size
        self.data = [None] * block_size  # 처음엔 비어있음
        self.filled_count = 0

    def append(self, token: str) -> bool:
        if self.filled_count < self.size:
            self.data[self.filled_count] = token
            self.filled_count += 1
            return True  # 저장 성공
        return False  # 꽉 차서 실패

    def __repr__(self):
        return f"[Block {self.block_id}]: {self.data}"

class VLLMMemoryManager:
    """
    vLLM의 핵심인 메모리 관리자 (OS 역할)
    """
    def __init__(self, total_blocks: int = 16, block_size: int = 4):
        self.block_size = block_size
        # 1. 물리적 메모리 공간 생성 (GPU VRAM)
        self.physical_memory = [PhysicalBlock(i, block_size) for i in range(total_blocks)]
        
        # 2. 사용 가능한 블록 리스트 (Free List)
        # 실제로는 물리적으로 떨어져 있어도 상관없음을 보여주기 위해 섞습니다.
        self.free_blocks = list(range(total_blocks))
        random.shuffle(self.free_blocks) 
        
        # 3. 요청별 블록 매핑 테이블 (Request ID -> List[Physical Block IDs])
        self.block_tables: Dict[str, List[int]] = {}

    def allocate_block(self) -> Optional[int]:
        """빈 물리 블록 하나를 가져옵니다."""
        if not self.free_blocks:
            return None # OOM (Out of Memory)
        return self.free_blocks.pop()

    def append_token(self, request_id: str, token: str):
        """
        핵심 로직: 토큰을 생성해서 KV Cache에 추가하는 과정
        """
        # 1. 해당 요청의 블록 테이블이 없으면 생성
        if request_id not in self.block_tables:
            print(f"--- [New Request] '{request_id}' 시작 ---")
            self.block_tables[request_id] = []

        # 2. 현재 사용 중인 마지막 물리 블록 확인
        current_block_id = None
        if self.block_tables[request_id]:
            current_block_id = self.block_tables[request_id][-1]
        
        # 3. 블록이 없거나 꽉 찼으면, 새로운 물리 블록 할당 (PagedAttention의 핵심!)
        if current_block_id is None or \
           self.physical_memory[current_block_id].filled_count >= self.block_size:
            
            new_block_id = self.allocate_block()
            if new_block_id is None:
                raise Exception("GPU Memory Full!")
            
            self.block_tables[request_id].append(new_block_id)
            current_block_id = new_block_id
            print(f"👉 [Alloc] 새로운 물리 블록 {current_block_id}번 할당됨")

        # 4. 물리 블록에 데이터 저장
        self.physical_memory[current_block_id].append(token)
        print(f"   [Write] 토큰 '{token}' -> 물리 블록 {current_block_id}번에 저장")

    def print_state(self, request_id: str):
        """현재 메모리 상태를 시각화해서 보여줍니다."""
        print(f"\n📊 [{request_id}] 의 PagedAttention 상태")
        table = self.block_tables.get(request_id, [])
        
        # 1. Logical View (사용자가 보는 문장)
        logical_text = []
        for block_id in table:
            block = self.physical_memory[block_id]
            logical_text.extend([t for t in block.data if t is not None])
        print(f"  1) 논리적 뷰 (Logical): {logical_text}")

        # 2. Block Table (매핑 정보)
        print(f"  2) 블록 테이블 (Mapping): {table}")

        # 3. Physical View (실제 저장 위치)
        print(f"  3) 물리적 뷰 (Physical):")
        for block_id in table:
            print(f"     {self.physical_memory[block_id]}")
        print("-" * 50)

print("✅ PagedAttention 시뮬레이터 클래스 준비 완료")

In [ ]:
# PagedAttention 시뮬레이션 실행
# LLM이 한 토큰씩 생성하는 상황을 흉내냅니다

# 1. 매니저 초기화 (Block Size = 4)
vllm_manager = VLLMMemoryManager(total_blocks=10, block_size=4)

# 2. 문장 생성 시뮬레이션
req_id = "User_A"
tokens = ["Deep", "Learn", "ing", "is", "very", "fun", "to", "study"]

print("🚀 토큰 생성 시뮬레이션 시작\n")
for token in tokens:
    vllm_manager.append_token(req_id, token)

# 3. 최종 상태 확인
vllm_manager.print_state(req_id)

### 실행 결과 해석

위 코드를 실행하면 다음과 같은 중요한 현상을 관찰할 수 있습니다:

#### 🔑 핵심 포인트 3가지

**1. 비연속성 (Non-contiguous)**
- 논리적으로는 `'is'` 다음에 `'very'`가 옵니다
- 하지만 물리적으로는 뜬금없는 블록 번호에 저장됩니다
- **의의**: 메모리의 빈 공간이 어디에 있든 상관없이 사용 가능 (메모리 단편화 해결)

**2. 동적 할당 (On-demand Allocation)**
- 블록이 꽉 찰 때(4개 토큰)만 새 블록 할당
- 만약 문장이 `'Deep Learning is'`에서 끝났다면? 두 번째 블록은 아예 할당되지 않음
- **의의**: 미리 잡아둔 공간의 낭비(Reserved slots waste)가 'Zero'

**3. 블록 테이블 (The Map)**
- vLLM은 `Block Table: [7, 2]` 같은 정보만 유지
- "5번째 토큰(`very`)을 가져와라" 요청 시:
  - `5 // 4 = 1` (인덱스 1번 블록 = Physical Block 2)
  - `5 % 4 = 1` (블록 내 1번째 오프셋 = 'very')
- **의의**: 이 계산이 매우 빠르기 때문에 성능 저하 없음

### 💡 왜 이게 중요한가?

PagedAttention이 GPU 메모리를 '테트리스' 하듯이 빈틈없이 채워 넣습니다. 이것이 vLLM이 다른 라이브러리보다 **압도적으로 빠르고 효율적인 이유**입니다!

---
## Part 2: vLLM 기본 사용법

### 2.1 모델 로딩

In [ ]:
# Step 1: vLLM 모델 초기화
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"

print("🚀 vLLM 모델 로딩 중...")
start_time = time.time()

llm = LLM(
    model=model_name,
    tensor_parallel_size=1,       # 단일 GPU 사용
    gpu_memory_utilization=0.9,   # GPU 메모리 90% 사용
    trust_remote_code=True,       # 커스텀 모델 코드 신뢰
)

load_time = time.time() - start_time
print(f"✅ 모델 로딩 완료! ({load_time:.2f}초)")

# Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"📝 Tokenizer 준비 완료")

### 2.2 간단한 추론 실습

In [ ]:
# Step 2: Sampling 파라미터 설정
sampling_params = SamplingParams(
    max_tokens=128,        # 최대 생성 토큰 수
    temperature=0.7,       # 샘플링 온도 (높을수록 다양성 증가)
    top_p=0.9,            # Nucleus sampling
    top_k=50,             # Top-k sampling
)

print("⚙️  Sampling 파라미터:")
print(f"  - Max Tokens: {sampling_params.max_tokens}")
print(f"  - Temperature: {sampling_params.temperature}")
print(f"  - Top-p: {sampling_params.top_p}")

In [ ]:
# Step 3: 추론 실행
prompts = [
    "Explain quantum computing in simple terms:",
    "What is the capital of France?",
    "Write a haiku about programming:",
]

print("🤖 추론 시작...")
start_time = time.time()

outputs = llm.generate(prompts, sampling_params)

inference_time = time.time() - start_time

print(f"\n✅ 추론 완료! ({inference_time:.2f}초)")
print(f"📊 처리량: {len(prompts) / inference_time:.2f} prompts/sec\n")

# 결과 출력
for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    token_count = len(output.outputs[0].token_ids)
    
    print(f"[프롬프트 {i+1}] {prompts[i]}")
    print(f"[생성 결과] {generated_text}")
    print(f"[토큰 수] {token_count}")
    print("-" * 60)

## Part 2.5: KV Cache 성능 비교 시뮬레이터

이제 **왜 KV Cache가 필요한지** 연산 횟수를 카운팅하여 직접 확인해봅시다!

이 시뮬레이터는 실제 행렬 연산 대신, **'연산을 몇 번 수행했는가'**를 측정합니다.

In [ ]:
class ModelSimulator:
    """LLM 추론 과정을 시뮬레이션"""
    def __init__(self):
        self.op_count = 0  # 연산 횟수 카운터

    def compute_token_representation(self, token):
        """
        토큰을 벡터로 변환 (Embedding + Q, K, V 생성)
        고비용 연산이라고 가정
        """
        self.op_count += 1
        return f"Vector({token})"

    def attention_calculation(self, current_token, past_vectors):
        """
        현재 토큰과 과거 토큰들 간의 관계 계산 (Attention)
        """
        self.op_count += len(past_vectors)

print("✅ ModelSimulator 준비 완료")

In [ ]:
# ==========================================
# 시나리오 1: KV Cache 없이 생성 (No Cache)
# ==========================================
print("🚫 [Scenario 1] KV Cache 미사용 (매번 처음부터 다시 계산)")
model_no_cache = ModelSimulator()
generated_tokens = []
input_prompt = ["Deep", "Learning", "is", "Fun"]

for step in range(5):
    current_context = input_prompt + generated_tokens
    
    # 매번 모든 토큰을 다시 계산 (비효율!)
    vectors = []
    for token in current_context:
        vec = model_no_cache.compute_token_representation(token)
        vectors.append(vec)
    
    model_no_cache.attention_calculation("New_Token", vectors)
    
    generated_tokens.append(f"Gen_{step}")
    print(f"  Step {step+1}: 문장 길이 {len(current_context)} -> 누적 연산: {model_no_cache.op_count}")

final_cost_no_cache = model_no_cache.op_count
print(f"\n총 연산 횟수: {final_cost_no_cache}")

In [ ]:
# ==========================================
# 시나리오 2: KV Cache 사용 (With Cache)
# ==========================================
print("\n✅ [Scenario 2] KV Cache 사용 (과거는 저장해두고 재사용)")
model_with_cache = ModelSimulator()
generated_tokens = []
kv_cache = []  # 이것이 바로 KV Cache!

# Prefill: 초기 프롬프트 처리 (한 번만)
print("  [Prefill] 초기 프롬프트 처리 중...")
for token in input_prompt:
    vec = model_with_cache.compute_token_representation(token)
    kv_cache.append(vec)

# Decode: 토큰 생성
for step in range(5):
    # 새로운 토큰 하나만 처리!
    current_vec = model_with_cache.compute_token_representation("New_Token")
    
    # 과거 cache + 현재 벡터로 attention
    model_with_cache.attention_calculation("New_Token", kv_cache)
    
    # 캐시에 저장
    kv_cache.append(current_vec)
    generated_tokens.append(f"Gen_{step}")
    
    print(f"  Step {step+1}: 캐시 크기 {len(kv_cache)} -> 누적 연산: {model_with_cache.op_count}")

final_cost_with_cache = model_with_cache.op_count
print(f"\n총 연산 횟수: {final_cost_with_cache}")

In [ ]:
# 최종 비교
print("\n" + "=" * 50)
print("📊 최종 연산 비용 비교 (낮을수록 빠름)")
print("=" * 50)
print(f"1. No Cache Cost  : {final_cost_no_cache} (O(N²) - 기하급수적 증가)")
print(f"2. With Cache Cost: {final_cost_with_cache} (O(N) - 선형적 증가)")
print(f"🚀 효율 향상: 약 {final_cost_no_cache / final_cost_with_cache:.1f}배 더 적은 연산!")
print("=" * 50)

### 결과 해석

**KV Cache**가 있고 없고의 차이가 엄청납니다!

- **No Cache**: O(N²) 비용. 문장이 길어지면 느려터짐
- **KV Cache**: O(N) 비용. 대신 메모리를 많이 씀 (메모리 ↔ 속도 교환)
- **vLLM**: KV Cache를 OS처럼 관리해서 메모리 낭비를 줄이고, 더 많은 KV Cache를 쑤셔 넣음!

---
## Part 3: Chat 형식 프롬프트

실전에서는 시스템 메시지, 유저 메시지, 어시스턴트 메시지를 조합한 chat 형식을 많이 사용합니다.

In [ ]:
def format_chat_messages(messages):
    """
    Chat 메시지를 프롬프트로 변환
    
    Args:
        messages: [{'role': 'system/user/assistant', 'content': '...'}, ...]
    """
    if tokenizer.chat_template:
        # Tokenizer에 chat template이 있으면 사용
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
    else:
        # 수동 포맷팅
        formatted = ""
        for msg in messages:
            role = msg['role'].capitalize()
            content = msg['content']
            formatted += f"{role}: {content}\n\n"
        
        if messages[-1]['role'] != 'assistant':
            formatted += "Assistant: "
        
        return formatted

print("✅ Chat 포맷팅 함수 준비 완료")

In [ ]:
# Text-to-SQL 예제
messages = [
    {
        'role': 'system',
        'content': 'You are a SQL expert. Convert natural language queries to SQL.'
    },
    {
        'role': 'user',
        'content': 'Find all users with age greater than 30'
    }
]

# 프롬프트 변환
prompt = format_chat_messages(messages)
print("📝 변환된 프롬프트:")
print(prompt)
print("-" * 60)

# 추론
sampling_params_sql = SamplingParams(
    max_tokens=64,
    temperature=0.1,  # SQL 생성은 낮은 temperature 사용
)

outputs = llm.generate([prompt], sampling_params_sql)
print("\n💡 생성된 SQL:")
print(outputs[0].outputs[0].text)

---
## Part 4: LoRA Adapter 사용하기

### 4.1 LoRA란?

**LoRA (Low-Rank Adaptation)**:
- 모든 파라미터를 fine-tuning하는 대신, 작은 adapter만 학습
- 메모리 효율적이고 빠른 학습 가능
- 여러 task별 adapter를 쉽게 교체 가능

### 4.2 vLLM에서 LoRA 사용하는 두 가지 방법

#### 방법 1: Runtime LoRA (동적 적용)
- 추론 시점에 adapter를 동적으로 로드
- 여러 adapter를 빠르게 교체 가능

In [ ]:
# Runtime LoRA 예제 (adapter가 있는 경우)
lora_adapter_path = "./lora_adapter"  # 실제 adapter 경로로 변경

if os.path.exists(lora_adapter_path):
    from vllm.lora.request import LoRARequest
    
    # LoRA 지원 모델 로드
    llm_with_lora = LLM(
        model=model_name,
        enable_lora=True,
        max_lora_rank=64,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,
        trust_remote_code=True,
    )
    
    # LoRA request 생성
    lora_request = LoRARequest("my_adapter", 1, lora_adapter_path)
    
    # 추론 (LoRA adapter 적용)
    outputs = llm_with_lora.generate(
        [prompt],
        sampling_params,
        lora_request=lora_request
    )
    
    print("✅ LoRA adapter 적용 완료")
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

#### 방법 2: Merged Model (사전 통합)
- LoRA weights를 base model에 미리 merge
- 더 빠른 추론 속도
- 배포에 유리

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

def merge_lora_to_base(base_model_name, lora_path, output_path):
    """
    LoRA adapter를 base model에 merge
    """
    print("🔄 LoRA merge 시작...")
    
    # 1. Base model 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        torch_dtype="auto"
    )
    
    # 2. LoRA adapter 로드
    model_with_lora = PeftModel.from_pretrained(base_model, lora_path)
    
    # 3. Merge
    merged_model = model_with_lora.merge_and_unload()
    
    # 4. 저장
    os.makedirs(output_path, exist_ok=True)
    merged_model.save_pretrained(output_path, safe_serialization=True)
    
    # Tokenizer도 저장
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.save_pretrained(output_path)
    
    print(f"✅ Merge 완료: {output_path}")
    return output_path

# 예제 (adapter가 있는 경우)
if os.path.exists(lora_adapter_path):
    merged_path = merge_lora_to_base(
        base_model_name=model_name,
        lora_path=lora_adapter_path,
        output_path="./merged_model"
    )
    
    # Merged 모델로 추론
    llm_merged = LLM(
        model=merged_path,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,
        trust_remote_code=True,
    )
    print("✅ Merged 모델로 추론 준비 완료")
else:
    print("⚠️ LoRA adapter 없음 - 건너뛰기")

---
## Part 5: 성능 벤치마크 - Transformers vs vLLM

vLLM이 얼마나 빠른지 직접 측정해봅시다!

In [ ]:
# 벤치마크 준비
test_prompts = [
    "You are a SQL expert. Convert this to SQL: Find all users",
    "You are a SQL expert. Convert this to SQL: Count employees",
    "You are a SQL expert. Convert this to SQL: Show top 10 sales",
    "You are a SQL expert. Convert this to SQL: Delete inactive accounts",
    "You are a SQL expert. Convert this to SQL: Update user emails",
]

max_tokens = 64

print(f"📊 벤치마크 설정")
print(f"  - 프롬프트 수: {len(test_prompts)}")
print(f"  - Max Tokens: {max_tokens}")

### 5.1 Transformers 벤치마크

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("🔵 Transformers 벤치마크 시작")
print("  모델 로딩 중...")

# 모델 로드
tf_tokenizer = AutoTokenizer.from_pretrained(model_name)
tf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
tf_model.eval()

# GPU 메모리 리셋
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# 추론
tf_start = time.time()
tf_total_tokens = 0
tf_first_token_latencies = []

for prompt in test_prompts:
    inputs = tf_tokenizer(prompt, return_tensors="pt").to(tf_model.device)
    
    # First token 측정
    ft_start = time.time()
    with torch.no_grad():
        outputs = tf_model(**inputs)
    first_token_time = time.time() - ft_start
    tf_first_token_latencies.append(first_token_time)
    
    # 전체 생성
    with torch.no_grad():
        generated = tf_model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
        )
    tf_total_tokens += generated.shape[1] - inputs.input_ids.shape[1]

tf_time = time.time() - tf_start
tf_peak_memory = torch.cuda.max_memory_allocated() / 1024 / 1024 if torch.cuda.is_available() else 0
tf_avg_first_token = sum(tf_first_token_latencies) / len(tf_first_token_latencies)

print(f"\n✅ Transformers 완료")
print(f"  ⏱️  총 시간: {tf_time:.2f}s")
print(f"  ⚡ First Token: {tf_avg_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {tf_total_tokens / tf_time:.2f}")
print(f"  💾 피크 메모리: {tf_peak_memory:.0f} MB")

# 메모리 정리
del tf_model
del tf_tokenizer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### 5.2 vLLM 벤치마크

In [ ]:
print("\n🟢 vLLM 벤치마크 시작")

# GPU 메모리 리셋
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()

# Sampling 파라미터
benchmark_params = SamplingParams(
    max_tokens=max_tokens,
    temperature=0.0,  # greedy decoding
)

# First token latency 측정
vllm_ft_start = time.time()
_ = llm.generate([test_prompts[0]], benchmark_params)
vllm_first_token = time.time() - vllm_ft_start

# 전체 추론
vllm_start = time.time()
vllm_outputs = llm.generate(test_prompts, benchmark_params)
vllm_time = time.time() - vllm_start

vllm_peak_memory = torch.cuda.max_memory_allocated() / 1024 / 1024 if torch.cuda.is_available() else 0
vllm_total_tokens = sum(len(out.outputs[0].token_ids) for out in vllm_outputs)

print(f"\n✅ vLLM 완료")
print(f"  ⏱️  총 시간: {vllm_time:.2f}s")
print(f"  ⚡ First Token: {vllm_first_token*1000:.2f}ms")
print(f"  🔥 Token/sec: {vllm_total_tokens / vllm_time:.2f}")
print(f"  💾 피크 메모리: {vllm_peak_memory:.0f} MB")

### 5.3 결과 비교

In [ ]:
print("\n" + "=" * 60)
print("📊 최종 비교 결과")
print("=" * 60)

# First Token Latency
print(f"\n⚡ First Token Latency:")
print(f"  Transformers: {tf_avg_first_token*1000:.2f}ms")
print(f"  vLLM:         {vllm_first_token*1000:.2f}ms")
ft_improvement = ((tf_avg_first_token - vllm_first_token) / tf_avg_first_token) * 100
print(f"  {'🚀 개선' if ft_improvement > 0 else '⚠️ 차이'}: {abs(ft_improvement):.1f}%")

# Token/sec
tf_tps = tf_total_tokens / tf_time
vllm_tps = vllm_total_tokens / vllm_time
print(f"\n🔥 Token/sec:")
print(f"  Transformers: {tf_tps:.2f} tokens/sec")
print(f"  vLLM:         {vllm_tps:.2f} tokens/sec")
print(f"  🚀 vLLM 향상: {vllm_tps / tf_tps:.2f}x")

# 총 시간
print(f"\n⏱️ 총 추론 시간:")
print(f"  Transformers: {tf_time:.2f}s")
print(f"  vLLM:         {vllm_time:.2f}s")
print(f"  🚀 속도 향상: {tf_time / vllm_time:.2f}x")

# 메모리
print(f"\n💾 피크 GPU 메모리:")
print(f"  Transformers: {tf_peak_memory:.0f} MB")
print(f"  vLLM:         {vllm_peak_memory:.0f} MB")
memory_diff = tf_peak_memory - vllm_peak_memory
if memory_diff > 0:
    print(f"  💡 메모리 절감: {memory_diff:.0f} MB ({memory_diff/tf_peak_memory*100:.1f}%)")
else:
    print(f"  📈 메모리 사용: {abs(memory_diff):.0f} MB 더 사용")

print("\n" + "=" * 60)

---
## Part 6: 종합 실습

### 최종 프로젝트: Text-to-SQL 시스템 구축

배운 내용을 종합하여 실전 Text-to-SQL 시스템을 만들어봅시다.

In [ ]:
class TextToSQLSystem:
    def __init__(self, model_name):
        self.llm = LLM(
            model=model_name,
            tensor_parallel_size=1,
            gpu_memory_utilization=0.9,
            trust_remote_code=True,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        self.system_prompt = """
You are an expert SQL query generator. Convert natural language questions to SQL queries.
Generate only the SQL query without explanations.
"""
    
    def generate_sql(self, question: str, schema_info: str = "") -> str:
        """자연어 질의를 SQL로 변환"""
        
        user_content = f"""
Database Schema:
{schema_info if schema_info else 'Standard database tables'}

Question: {question}

SQL Query:
"""
        
        messages = [
            {'role': 'system', 'content': self.system_prompt},
            {'role': 'user', 'content': user_content}
        ]
        
        prompt = format_chat_messages(messages)
        
        sampling_params = SamplingParams(
            max_tokens=128,
            temperature=0.1,
            stop=[";", "\n\n"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        sql_query = outputs[0].outputs[0].text.strip()
        
        return sql_query

# 시스템 초기화
sql_system = TextToSQLSystem(model_name)
print("✅ Text-to-SQL 시스템 준비 완료")

In [ ]:
# 테스트
schema = """
Tables:
- users (id, name, email, age, department_id)
- departments (id, name, budget)
- orders (id, user_id, product_id, amount, order_date)
"""

test_questions = [
    "Find all users older than 30",
    "Count employees in each department",
    "Show top 5 users by order amount",
    "List departments with budget over 100000",
]

print("🧪 Text-to-SQL 테스트\n")

for question in test_questions:
    print(f"❓ Question: {question}")
    sql = sql_system.generate_sql(question, schema)
    print(f"💡 Generated SQL: {sql}")
    print("-" * 60)